# Adding a user-defined decision module to the turtles: The boids model

The [boids](https://en.wikipedia.org/wiki/Boids) (bird-oid) model has been invented by [Craig Reynolds](https://en.wikipedia.org/wiki/Craig_Reynolds_(computer_graphics)) in 1986 to simulate the flocking behavior of birds. It is based on 3 principles:
    
* separation: boids tend to avoid other boids that are too close,

* alignment: boids tend to align their velocity to boids that are not too close and not too far away,

* cohesion: bois tend to move towards boids that are too far away.

While these rules are essentially heuristic, they can be implemented defining three areas (repulsion, orientation, attraction) for each principle. 

* Boids change their orientation to get away from other boids in the repulsion area,

* Boids change their orientation and speed to match those of other boids in the orientation area,

* Boids change their orientation to get to other boids in the attraction area.

## Grab dependencies

In [1]:
#!ruby
#@repository("~/.m2/repository/")
#@dependency(group="fr.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")

java_import 'fr.lgi2a.similar2logo.lib.tools.random.PRNG'
java_import 'fr.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel'
java_import 'fr.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters'
java_import 'fr.lgi2a.similar.microkernel.AgentCategory'
java_import 'fr.lgi2a.similar.microkernel.LevelIdentifier'
java_import 'fr.lgi2a.similar.microkernel.SimulationTimeStamp'
java_import 'fr.lgi2a.similar.microkernel.ISimulationModel'
java_import 'fr.lgi2a.similar.microkernel.agents.IGlobalState'
java_import 'fr.lgi2a.similar.microkernel.agents.ILocalStateOfAgent'
java_import 'fr.lgi2a.similar.microkernel.agents.IPerceivedData'
java_import 'fr.lgi2a.similar.microkernel.influences.InfluencesMap'
java_import 'fr.lgi2a.similar.microkernel.levels.ILevel'
java_import 'fr.lgi2a.similar2logo.kernel.initializations.AbstractLogoSimulationModel'
java_import 'fr.lgi2a.similar2logo.kernel.model.LogoSimulationParameters'
java_import 'fr.lgi2a.similar2logo.kernel.model.Parameter'
java_import 'fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory'
java_import 'fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory'
java_import 'fr.lgi2a.similar2logo.kernel.model.influences.ChangeDirection'
java_import 'fr.lgi2a.similar2logo.kernel.model.influences.ChangeSpeed'
java_import 'fr.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList'
java_import 'fr.lgi2a.similar2logo.lib.model.ConeBasedPerceptionModel'
java_import 'fr.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner'
java_import 'fr.lgi2a.similar2logo.lib.tools.math.MeanAngle'
java_import 'fr.lgi2a.similar2logo.kernel.tools.MathUtil'

Resolving dependency: fr.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact com.sparkjava#spark-core;2.7.2!spark-core.jar(bundle)
Preparing to download artifact org.slf4j#slf4j-simple;1.7.13!slf4j-simple.jar
Preparing to download artifact org.apache.commons#commons-math3;3.6.1!commons-math3.jar
Preparing to download artifact fr.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to download artifact fr.lgi2a#similar-microKernel-commonLibs;1.0-SNAPSHOT!similar-microKernel-commonLibs.jar
Preparing to download artifact fr.lgi2a#similar-extendedKernel;1.0-SNAPSHOT!similar-extendedKernel.jar
Preparing to download artifact fr.lgi2a#similar-extendedKernel-extendedLibs;1.0-SNAPSHOT!similar-extendedKernel-extendedLibs.jar
Preparing to download artifact o

Java::FrLgi2aSimilar2logoKernelTools::MathUtil


## Model parameters

The model parameters and their default values are defined.

In [2]:
class BoidsSimulationParameters < LogoSimulationParameters
  
  attr_accessor :repulsionDistance, :attractionDistance, :orientationDistance, :repulsionWeight, :orientationWeight, :attractionWeight, :maxInitialSpeed, :minInitialSpeed, :perceptionAngle, :nbOfAgents, :maxAngle
  
  def initialize
    
    @repulsionDistance = 6
  
    @attractionDistance = 14
  
    @orientationDistance  = 10
    
    @repulsionWeight = 1
    
    @orientationWeight = 1
    
    @attractionWeight = 1
  
    @maxInitialSpeed = 2
  
    @minInitialSpeed = 1
  
    @perceptionAngle = Math::PI
  
    @nbOfAgents = 200
 
    @maxAngle = Math::PI/8
  end
  
end

No Outputs

## Decision model

The decision model consists in changing the direction and speed of the boids according to the previously described rules.
To define a decision model, the modeler must define an object that extends `AbstractAgtDecisionModel` and implement the `decide` method.

In [3]:
class BoidDecisionModel < AbstractAgtDecisionModel
  
  def initialize(parameters)
    super(LogoSimulationLevelList::LOGO)
    @parameters = parameters
  end
  
  def decide(
    timeLowerBound,
    timeUpperBound,
    globalState,
    publicLocalState,
    privateLocalState,
    perceivedData,
    producedInfluences
  )
    if !perceivedData.getTurtles.empty?
      meanAngle = MeanAngle.new
      orientationSpeed = 0
      nbOfTurtlesInOrientationArea = 0
      perceivedData.getTurtles.each do |perceivedTurtle|
        if perceivedTurtle  != publicLocalState
          if perceivedTurtle.getDistanceTo <= @parameters.repulsionDistance
            meanAngle.add(publicLocalState.getDirection - perceivedTurtle.getDirectionTo, @parameters.repulsionWeight)
          elsif perceivedTurtle.getDistanceTo <= @parameters.orientationDistance
            meanAngle.add(perceivedTurtle.getContent.getDirection - publicLocalState.getDirection, @parameters.orientationWeight)
            orientationSpeed+=perceivedTurtle.getContent.getSpeed - publicLocalState.getSpeed
            nbOfTurtlesInOrientationArea+=1
          elsif perceivedTurtle.getDistanceTo <= @parameters.attractionDistance
            meanAngle.add(perceivedTurtle.getDirectionTo- publicLocalState.getDirection, @parameters.attractionWeight)
          end
        end
      end
      dd = meanAngle.value
      if !MathUtil::areEqual(dd, 0)
        if dd > @parameters.maxAngle
          dd = @parameters.maxAngle
        elsif dd<-@parameters.maxAngle
          dd = -@parameters.maxAngle
        end
        producedInfluences.add(
          ChangeDirection.new(
           timeLowerBound,
           timeUpperBound,
           dd,
           publicLocalState
         )
       )
     end
     if nbOfTurtlesInOrientationArea > 0
        orientationSpeed /= nbOfTurtlesInOrientationArea
        producedInfluences.add(
          ChangeSpeed.new(
            timeLowerBound,
            timeUpperBound,
            orientationSpeed,
            publicLocalState
          )
        )
     end
    end
  end
end

No Outputs

## The simulation model

In the simulation model defined in our example, boids are randomly located in the environment with a random orientation and speed.

In [4]:
class BoidsSimulationModel < AbstractLogoSimulationModel
  def generateAgents(p, levels)
     result =  AgentInitializationData.new
     p.nbOfAgents.times do
      result.getAgents.add(
        TurtleFactory::generate(
         ConeBasedPerceptionModel.new(p.attractionDistance,p.perceptionAngle,true,false,false),
         BoidDecisionModel.new(p),
         AgentCategory.new("b", TurtleAgentCategory::CATEGORY),
         Math::PI-PRNG::get.randomAngle,
         p.minInitialSpeed + PRNG::get.randomDouble*(
           p.maxInitialSpeed-p.minInitialSpeed
         ),
         0,
         PRNG::get.randomDouble*p.gridWidth,
         PRNG::get.randomDouble*p.gridHeight
       )
      )
    end
    return result
  end
end

No Outputs

## Launch the HTML runner

Finally, we launch and configure the HTML runner.

In [5]:
runner = Similar2LogoHtmlRunner.new
runner.config.setExportAgents(true)
runner.initializeRunner(BoidsSimulationModel.new(BoidsSimulationParameters.new))
runner.showView

No Outputs